In [3]:
!pip install transformers accelerate bitsandbytes sentence-transformers wikipedia duckduckgo-search


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 51.4 MB/s eta 0:00:00
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=bb580d81d813ea787d02570ee61a29b40fe3d33248d89431d8115174907b8197
  Stored in directory: /root/.cache/pip/wheels/63/47/7c/a9688349aa74d228ce0a9023229c6c0ac52ca2a40fe87679b8
Successfully built wikipedia


In [4]:
!pip install -U bitsandbytes
!pip install -U transformers accelerate


In [5]:
pip install -U bitsandbytes

In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig

model_name = "mistralai/Mistral-7B-Instruct-v0.2"

# Quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

# LLM pipeline
llm = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.2
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Device set to use cuda:0


In [7]:
def analyze_article_json(article):
    import json, re
    """
    Analyzes article bias, credibility, and emotional manipulation using Mistral.
    Returns parsed JSON or error.
    """
    prompt = f"""
    You are a media analysis expert.
    Analyze the following news article and output ONLY valid JSON with real values.

    Fields:
    - credibility_score: integer (0-100)
    - Political bias: "Left", "Right", "Center", or "Unknown"
    - emotional_tone: "Neutral", "Emotional", or "Manipulative"
    - persuasive_techniques: list of manipulative techniques (if any)
    - bias_summary: short summary of bias
    - reasoning: short reasoning

    Article:
    {article}

    Output ONLY JSON (no explanations, no markdown).
    """

    result = llm(prompt)[0]["generated_text"]

    # Extract JSON
    match = re.search(r"\{.*\}", result, re.DOTALL)
    if match:
        try:
            return json.loads(match.group())
        except Exception as e:
            return {"error": f"JSON parse error: {e}", "raw_output": result}
    else:
        return {"error": "No JSON found", "raw_output": result}


In [33]:
article = """In yet another display of incompetence, the current government has once again failed the people. Despite endless promises to tackle inflation and improve living conditions, ordinary citizens continue to suffer under the crushing weight of rising prices and job losses. The Prime Minister, who seems more interested in attending photo ops and luxury events than addressing real issues, has repeatedly ignored the pleas of small business owners and working families.

Experts have warned for months that the administration’s reckless spending and weak policies would lead to disaster, but the government has chosen to turn a blind eye. Meanwhile, opposition leaders have been tirelessly working to expose the corruption and mismanagement that have plagued the ruling party since the beginning of its term. Yet, the state-controlled media continues to protect those in power, painting a false picture of progress and stability.

Citizens across the country are growing increasingly frustrated as basic necessities become unaffordable. Mothers struggle to feed their children, young people are leaving the country in search of better opportunities, and hospitals are running out of essential medicines — all while the government insists everything is under control.

It’s becoming painfully clear that the administration has lost touch with reality. Instead of addressing the crisis, ministers have spent millions on propaganda campaigns to convince people that the economy is booming. But the truth is undeniable: unemployment is at record highs, corruption is rampant, and trust in public institutions is at an all-time low.

The people deserve better than empty speeches and false promises. Until this government is replaced by competent leaders who actually care about citizens, the nation will continue to spiral deeper into chaos. The upcoming elections may be the last chance to save the country from complete collapse.
"""
import json

# Example usage
bias_json = analyze_article_json(article)
print(json.dumps(bias_json, indent=2))


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{
  "credibility_score": 85,
  "political_bias": "Left",
  "emotional_tone": "Emotional",
  "persuasive_techniques": [
    "Fear",
    "Appeal to emotions"
  ],
  "bias_summary": "The article expresses a clear left-leaning bias and uses emotional language to appeal to readers' frustration and concern for the well-being of citizens.",
  "reasoning": "The text criticizes the current government's incompetence and lack of concern for citizens, while praising opposition leaders and expressing hope for change through elections. The emotional tone is evident in the use of words like 'crushing weight,' 'growing increasingly frustrated,' and 'complete collapse.'"
}


In [34]:

import os
# Define output folder and filename
output_folder = "bias_results"
os.makedirs(output_folder, exist_ok=True)

# Optional: create a unique filename per article
filename = "article_bias_result.json"
output_path = os.path.join(output_folder, filename)

# Save to JSON file
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(bias_json, f, indent=2, ensure_ascii=False)

print(f"✅ JSON result saved to: {output_path}")

✅ JSON result saved to: bias_results/article_bias_result.json


In [35]:
def llm_decide_bias(bias_json):
    import json, re

    prompt = f"""
You are a bias evaluation expert.
You will receive a JSON analysis of a news article from another model.

Decide whether the article is "Biased" or "Not Biased" based on the data.

Rules:
- "Biased" if:
    • political_bias is not "Center" and not "Unknown"
    • emotional_tone is "Emotional" or "Manipulative"
    • persuasive_techniques list is not empty
    • credibility_score < 60
- Otherwise: "Not Biased"

Return ONLY valid JSON in this exact format:
{{
  "bias_decision": "Biased" or "Not Biased",
  "reasoning": "short justification"
}}

Input JSON:
{json.dumps(bias_json, indent=2)}

Output ONLY JSON — no explanations, no markdown, no ``` backticks.
"""

    result = llm(prompt)[0]["generated_text"]

    # Extract the last JSON object
    matches = re.findall(r"\{[\s\S]*?\}", result)
    if matches:
        json_text = matches[-1]
        try:
            return json.loads(json_text)
        except Exception as e:
            return {"error": f"JSON parse error: {e}", "raw_output": result}
    else:
        return {"error": "No JSON found", "raw_output": result}


In [36]:
print("\n🤖 Step 2: LLM deciding bias...")

bias_decision = llm_decide_bias(bias_json)
print(json.dumps(bias_decision, indent=2))


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



🤖 Step 2: LLM deciding bias...
{
  "bias_decision": "Biased",
  "reasoning": "The article expresses a clear left-leaning bias and uses emotional language to appeal to readers' frustration and concern for the well-being of citizens. Persuasive techniques include Fear and Appeal to emotions."
}


In [32]:
with open("bias_results/final_bias_decision.json", "w", encoding="utf-8") as f:
  json.dump(final_decision, f, indent=2, ensure_ascii=False)
print("✅ Final decision saved to bias_results/final_bias_decision.json")

✅ Final decision saved to bias_results/final_bias_decision.json
